# More JOIN operations

![rel](https://sqlzoo.net/w/images/5/50/Movie2-er.png)

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. 1962 movies

List the films where the **yr** is 1962 [Show **id, title**]

In [2]:
movie <- dbReadTable(con, 'movie')
casting <- dbReadTable(con, 'casting')
actor <- dbReadTable(con, 'actor')

In [3]:
movie %>% filter(yr==1962) %>% select(id, title)

id,title
<int>,<chr>
10212,A Kind of Loving
10329,A Symposium on Popular Songs
10347,A Very Private Affair (Vie PrivÃ©e)
10648,An Autumn Afternoon
10868,Atraco a las tres
11006,Barabbas
11053,Battle Beyond the Sun (ÐÐµÐ±Ð¾ Ð·Ð¾Ð²ÐµÑ‚)
11199,Big and Little Wong Tin Bar
11230,Billy Budd


## 2. When was Citizen Kane released?

Give year of 'Citizen Kane'.

In [4]:
movie %>% 
    filter(title=='Citizen Kane') 
    %>% select(yr)

yr
<int>
1941


## 3. Star Trek movies

List all of the Star Trek movies, include the **id**, **title** and **yr** (all of these movies include the words Star Trek in the title). Order results by year.

In [5]:
movie %>% 
    filter(str_detect(title, 'Star Trek')) %>% 
    select(id, title, yr) %>% 
    arrange(yr)

id,title,yr
<int>,<chr>,<int>
17772,Star Trek: The Motion Picture,1979
17775,Star Trek II: The Wrath of Khan,1982
17776,Star Trek III: The Search for Spock,1984
17777,Star Trek IV: The Voyage Home,1986
17779,Star Trek V: The Final Frontier,1989
17780,Star Trek VI: The Undiscovered Country,1991
17774,Star Trek Generations,1994
17770,Star Trek: First Contact,1996
17771,Star Trek: Insurrection,1998


## 4. id for actor Glenn Close

What **id** number does the actor 'Glenn Close' have?

In [6]:
actor %>%
    filter(name=='Glenn Close') %>%
    select(id)

id
<int>
140


## 5. id for Casablanca

What is the **id** of the film 'Casablanca'

In [7]:
movie %>% 
    filter(title=='Casablanca') %>% 
    select(id)

id
<int>
11768


## 6. Cast list for Casablanca

Obtain the cast list for 'Casablanca'.

> _what is a cast list?_  
> The cast list is the names of the actors who were in the movie.

Use **movieid=11768**, (or whatever value you got from the previous question)

In [8]:
casting %>% 
    filter(movieid==11768) %>% 
    inner_join(actor, by=c(actorid="id")) %>%
    select(name)

name
<chr>
Peter Lorre
John Qualen
Madeleine LeBeau
Jack Benny
Dan Seymour
Norma Varden
Ingrid Bergman
Conrad Veidt
Leon Belasco


## 7. Alien cast list

Obtain the cast list for the film 'Alien'

In [9]:
movie %>% 
    filter(title=='Alien') %>% 
    inner_join(casting, by=c(id="movieid")) %>%
    inner_join(actor, by=c(actorid="id")) %>%
    select(name)

name
<chr>
John Hurt
Sigourney Weaver
Yaphet Kotto
Harry Dean Stanton
Ian Holm
Tom Skerritt
Veronica Cartwright


## 8. Harrison Ford movies

List the films in which 'Harrison Ford' has appeared

In [10]:
actor %>% 
    filter(name=='Harrison Ford') %>%
    inner_join(casting, by=c(id="actorid")) %>%
    left_join(movie, by=c(movieid="id")) %>%
    select(title)

title
<chr>
A Hundred and One Nights
Air Force One
American Graffiti
Apocalypse Now
Clear and Present Danger
Cowboys & Aliens
Crossing Over
Dead Heat on a Merry-Go-Round
Extraordinary Measures


## 9. Harrison Ford as a supporting actor

List the films where 'Harrison Ford' has appeared - but not in the starring role. [Note: the ord field of casting gives the position of the actor. If ord=1 then this actor is in the starring role]

In [11]:
actor %>% 
    filter(name=='Harrison Ford') %>%
    inner_join(casting, by=c(id="actorid")) %>%
    filter(ord>1) %>%
    left_join(movie, by=c(movieid="id")) %>%
    select(title)

title
<chr>
A Hundred and One Nights
American Graffiti
Apocalypse Now
Cowboys & Aliens
Dead Heat on a Merry-Go-Round
Extraordinary Measures
Force 10 From Navarone
Hawthorne of the U.S.A.
Jimmy Hollywood


## 10. Lead actors in 1962 movies

List the films together with the leading star for all 1962 films.

In [12]:
actor %>%
    inner_join(casting, by=c(id="actorid")) %>%
    filter(ord==1) %>%
    left_join(movie, by=c(movieid="id")) %>%
    filter(yr==1962) %>%
    select(title, name) %>%
    arrange(title)

title,name
<chr>,<chr>
A Kind of Loving,Alan Bates
A Symposium on Popular Songs,Paul Frees
A Very Private Affair (Vie PrivÃ©e),Brigitte Bardot
An Autumn Afternoon,Chishu Ryu
Atraco a las tres,JosÃ© Luis LÃ³pez VÃ¡zquez
Barabbas,Anthony Quinn
Battle Beyond the Sun (ÐÐµÐ±Ð¾ Ð·Ð¾Ð²ÐµÑ‚),Aleksandr Shvorin
Big and Little Wong Tin Bar,Jackie Chan
Billy Budd,Terence Stamp


## 11. Busy years for Rock Hudson

Which were the busiest years for 'Rock Hudson', show the year and the number of movies he made each year for any year in which he made more than 2 movies.

In [13]:
actor %>%
    filter(name=='Rock Hudson') %>%
    inner_join(casting, by=c(id="actorid")) %>%
    inner_join(movie, by=c(movieid="id")) %>%
    group_by(yr) %>% 
    summarise(n=n()) %>%
    filter(n>2) %>% 
    arrange(desc(n))

yr,n
<int>,<int>
1953,5
1961,3


## 12. Lead actor in Julie Andrews movies

List the film title and the leading actor for all of the films 'Julie Andrews' played in.

> _Did you get "Little Miss Marker twice"?_   
> Julie Andrews starred in the 1980 remake of Little Miss Marker and not the original(1934).
>
> Title is not a unique field, create a table of IDs in your subquery

In [14]:
movie %>% 
    inner_join(casting, by=c(id="movieid")) %>%
    inner_join(actor, by=c(actorid="id")) %>%
    filter(ord==1 & id %in% (
        movie %>% 
        inner_join(casting, by=c(id="movieid")) %>%
        inner_join(actor, by=c(actorid="id")) %>%
        filter(name=='Julie Andrews') %>% 
        select(id) %>% 
        pull)
    ) %>% 
    select(title, name)

title,name
<chr>,<chr>
10,Dudley Moore
Darling Lili,Julie Andrews
Despicable Me,Steve Carell
Duet for One,Julie Andrews
Hawaii,Julie Andrews
Little Miss Marker,Walter Matthau
Mary Poppins,Julie Andrews
Relative Values,Julie Andrews
Shrek the Third,Mike Myers


## 13. Actors with 15 leading roles

Obtain a list, in alphabetical order, of actors who've had at least 15 **starring** roles.

In [15]:
casting %>% 
    filter(ord==1) %>% 
    group_by(actorid) %>% 
    summarise(n=n()) %>%
    filter(n>=15) %>% 
    inner_join(actor, by=c(actorid="id")) %>%
    select(name) %>%
    arrange(name)

name
<chr>
Adam Sandler
Al Pacino
Anthony Hopkins
Antonio Banderas
Arnold Schwarzenegger
Barbara Stanwyck
Basil Rathbone
Ben Affleck
Bette Davis


## 14.
List the films released in the year 1978 ordered by the number of actors in the cast, then by title.

In [16]:
movie %>% 
    filter(yr==1978) %>% 
    inner_join(casting, by=c(id="movieid")) %>%
    group_by(id, title) %>% 
    summarise(actors=n()) %>%
    ungroup() %>%
    select(title, actors) %>% 
    arrange(title)

title,actors
<chr>,<int>
A Night Full of Rain,19
A Wedding,19
Alexandria... Why?,15
Almost Summer,13
American Boy: A Profile of Steven Prince,3
American Hot Wax,27
An Unmarried Woman,13
Attack of the Killer Tomatoes,7
Autumn Sonata,11


## 15.

List all the people who have worked with 'Art Garfunkel'.

In [17]:
actor %>%
    inner_join(casting, by=c(id="actorid")) %>%
    filter(movieid %in% (
        actor %>% 
        inner_join(casting, by=c(id="actorid")) %>%
        filter(name=='Art Garfunkel') %>% 
        select(movieid) %>%
        pull)
    ) %>% 
    filter(name != 'Art Garfunkel') %>% 
    select(name)

name
<chr>
Beverly Johnson
Elio Fiorucci
Robert DoQui
Donald Trump
Michael York
Ron Jeremy
Cindy Crawford
Heidi Klum
Mark Ruffalo


In [18]:
dbDisconnect(con)